In [1]:
import subprocess
import re

In [2]:
subprocess.run(["pdf2txt.py", "test1.pdf", "-o","test1.txt"])

CompletedProcess(args=['pdf2txt.py', 'test1.pdf', '-o', 'test1.txt'], returncode=0)

In [14]:
result = []
result1 = []
with open('test1.txt', 'r') as f:
    flag = 0
    flag2 = 0
    cc = 0
    tt = 0
    cclist = []
    ttlist = []
    ttlistX =[]
    ttlistY = []
    cct=""
    ttt=""
    termC = 0
    termC_ = 0
    count = 0
    for c,line in enumerate(f):
        #1bug where "IV." might appear in a sentence 
        if line.find("IV.")!=-1 and c > 100:
            flag = 1
            continue
        if flag == 1:
            #get court construction
            if (cc == 0) and re.search("Court.*Construction$",line) and (flag2 == 1):
                cct+=line
                cc = 1
                flag2 = 0
            elif cc >= 1:
                if (cc >= 2) and re.search("^\s*[A-Z0-9]{1,3}\.\s*",line):
                    cc = 0
                    tt = 0
                    count = 0
                    ttlist.append(ttlistX)
                    ttlistX=[]
                    cclist.append(" ".join((cct.split())[3:]))
                    cct=""
                elif (cc >= 2) and re.search("^\s*\n",line):
                    cc += 1
                    cct+=line
                elif (cc == 1) and re.search("\w\.\W*\s*\n",line):
                    cc += 1
                    cct+=line
                else:
                    cc = 1
                    cct+=line
            #--- end court construction ---
            #get table
            if (tt == 0) and re.search("^\s*[A-Z]{1,2}\.",line):
                if line.find("“") != -1:
                    flag2 = 1
                    tt = 1
                    termC_ = len([x for x in line.split() if (x.find("“") != -1)])
                    termC = 3*termC_
            elif (tt == 1) and (count < termC):
                if (count < termC_) and re.search("\w+",line):
                    if line.find("Claim") != -1:
                        if count == 0:
                            ttt = ttt.split("“")
                            c = 0
                            s = ''
                            temp = ttt[0].strip().split()
                            while c < (len(temp)):
                                if c%2 != 0:
                                    s += temp[c]
                                    ttlistX.append([s])
                                    s =''
                                else:
                                    s += temp[c] + " "
                                c += 1
                            ttt = "“" + ttt[-1] + " "
                        count += 1
                        ttt += " ".join(line.split())
                        ttlistY.append(ttt)
                        ttt = ""
                    else:
                        ttt += " ".join(line.split()) + " "
                elif (count >= termC_):
                    if re.search("\w", line):
                        ttt += " ".join(line.split()) + " "
                    elif re.search("\W", line) and (ttt != ""):
                        count += 1
                        ttt = ttt.strip()
                        ttlistY.append(ttt)
                        ttt = ""
                if (count > 0) and (len(ttlistY)==termC_) and (len(ttlistX)==3):
                    for x in ttlistY:
                        ttlistX[(count//termC_)-1].append(x)
                    ttlistY = [] 
            #--- end get table ---
            if line.find("V.")!=-1:
                break

    result.append(ttlist)
    result.append(cclist)
    #combine table
    for i in ttlist[1:]:
        for c,j in enumerate(i):
            for c1,k in enumerate(j):
                    ttlist[0][c].append(k)
    result1.append(ttlist[0])
    result1.append(cclist)

In [158]:
import pickle

In [160]:
with open('test1.pkl', 'wb') as f:
    pickle.dump(result, f)

In [162]:
with open('test1.pkl', 'rb') as f:
    r = pickle.load(f)


In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame(result[0][0]).T
df

,0,1,2
0,Disputed Term,Plaintiff’s Proposal,Defendant’s Proposal
1,"“replacing” (Claim 1, 2)","[No construction necessary]; or, if the Court ...","“using a noticeably gradual effect, taking the..."
2,“displacing” (Claim 2),"[No construction necessary]; or, if the Court ...","“using a noticeably gradual effect, taking the..."
3,“contracting” (Claim 2),"[No construction necessary]; or, if the Court ...","“using a noticeably gradual effect, decreasing..."
4,Disputed Term,Plaintiff’s Proposal,Defendant’s Proposal
5,“indication of speech uttered” (Claim 1),"[No construction necessary]; or, if the Court ...",“any momentary speech”


In [184]:
df.to_csv("test1_1.csv",index=False)

In [15]:
df1 = pd.DataFrame(result1[1])

In [185]:
df1.to_csv("test1_2.csv",index=False)

In [16]:
df1

,0
0,"For the reasons set forth above, the Court con..."
1,"For the reasons set forth above, the Court con..."
